### Remplacement des locuteurs "non_available" par leur locuteur sélectionné dans Prodigy

In [2]:
import os
import json

#### Fichier json

In [3]:
json_file = "/vol/work1/bergoend/characters.jsonl"

with open(json_file, 'r') as json_file:
    json_list = list(json_file)
    
print("Nombre d'annotations faites :", len(json_list))
    

Nombre d'annotations faites : 457


## Se baser sur la similarité du texte dans la BDD avec celle des phrases coupées par l'alignement forcé ET leur temps de début

Afin de placer le bon personnage devant le bon texte on se base sur la similarité du texte entre l'alignement forcé et la correction dans la BDD Prodigy. Mais, puisqu'une même phrase peut être prononcée à différents moments de l'épisode (et parfois par différents locuteurs), on cherche à remplacer les phrases similaires avec des temps de début et de fins similaires. De cette façon les bons locuteurs sont placés devant les phrases prononcées.

In [8]:
import spacy
from forced_alignment import ForcedAlignment

#### Charger l'alignement forcé

In [9]:
forced_alignment = ForcedAlignment()

In [10]:
# test
transcript_file = '24.Season01.Episode01'

# path to forced alignment
if os.path.isfile(f"../pyannote-db-plumcot/Plumcot/data/24/forced-alignment/{transcript_file}.aligned"):
    aligned = f"../pyannote-db-plumcot/Plumcot/data/24/forced-alignment/{transcript_file}.aligned"
    print(aligned)
else:
    print("No aligned file for")
    aligned = ""
        
# load forced alignment        
transcript = forced_alignment(aligned)      
sentences = list(transcript.sents)

../pyannote-db-plumcot/Plumcot/data/24/forced-alignment/24.Season01.Episode01.aligned


In [11]:
print("Nombres de phrases dans l'alignement forcé :", len(sentences))

Nombres de phrases dans l'alignement forcé : 675


## Recherche des similarités

##### 1. créer un dictionnaire pour l'alignement forcé et un pour les annotations
##### 2. Key : (temps_début,temps_fin)
##### 3. Val : (locuteur, phrase)
### But : trouver les clefs temps communes entre les deux dictionnaires, puis obtenir quel est le locuteur pour la phrase à temps_début, temps_fin

In [12]:
# spacy dictionary
spacy_dic = {(str(sentence._.start_time), str(sentence._.end_time)) : (sentence._.speaker, sentence) for sentence in sentences}

In [13]:
print("Nombre de phrases dans l'alignement forcé :", len(spacy_dic))
#print(spacy_dic)

Nombre de phrases dans l'alignement forcé : 675


### Créer dictionnaire avec les annotations

In [14]:
# annotations dictionary
correction_dic = {}
for dictionary in json_list:
    result = json.loads(dictionary)
    # find the annotated character
    if result['answer'] == 'accept':
        correction_dic[(result['start_time'], result['end_time'])]= (result['accept'][0], result['text'])
    # no annotation for the speaker = "not_available"
    if result['answer'] == 'ignore':
        correction_dic[(result['start_time'], result['end_time'])]= ("not_available", result['text'])

In [15]:
print("Nombre de phrases annotées :", len(correction_dic))
#print(correction_dic)

Nombre de phrases annotées : 457


#### Annotations restantes

In [17]:
not_annotated = { k : spacy_dic[k] for k in set(spacy_dic) - set(correction_dic) }
print("Annotations restantes :", len(not_annotated))

Annotations restantes : 218


##### Garder les éléments qui n'ont pas été annotés pour les écrire dans le fichier final

In [18]:
# merge annotations and not annotated elements
final = {**correction_dic, **not_annotated}

# sort by times
final = {key: val for key, val in sorted(final.items(), key=lambda item: float(item[0][0])) }
print("Nombre d'annotations à écrire dans le fichier final :", len(final))

Nombre d'annotations à écrire dans le fichier final : 675


### Comparaison de l'ancien alignement avec le nouveau

Vérification visuelle, voir captures d'écran

In [61]:
with open('24.Season01.Episode01_spacy.txt', "w", encoding='utf-8') as w:
    spacy = w
    for sentence in sentences:
        spacy.write(str(sentence._.speaker) + ' ' + str(sentence) + '\n')

#### Ecriture du nouveau fichier de transcription 
(Nom à modifier), Ecriture du locuteur et du texte prononcé

In [ ]:
with open('24.Season01.Episode01_correction.txt', "w", encoding='utf-8') as f:
    correction = f
    for i, j in final.items():
        # write speaker's name and sentence
        correction.write(str(j[0]) + ' ' + str(j[1]) + "\n")